In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import confusion_matrix, accuracy_score, zero_one_loss

%config InlineBackend.figure_format='svg'
%matplotlib inline

In [12]:
covertype_path = './covtype/covtype.data'

names = [
    'Elevation',
    'Aspect',
    'Slope',
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Hillshade_9am',
    'Hillshade_Noon',
    'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points',
]

Wilderness_Area = []
for i in range(4):
    Wilderness_Area.append('Wilderness_Area_{}'.format(i+1))

Soil_Type = []
for i in range(40):
    Soil_Type.append('Soil_Type_{}'.format(i+1))

names.extend(Wilderness_Area + Soil_Type + ['Cover_Type'])
covertype_df = pd.read_csv(covertype_path, names=names)

In [13]:
covertype_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [14]:
X = covertype_df.drop('Cover_Type',axis=1)
y = covertype_df['Cover_Type']

X_train, X_test, y_train, y_test = train_test_split(X.as_matrix(), y.as_matrix(), test_size=0.75, random_state=42)

In [19]:
train, test = train_test_split(covertype_df.as_matrix(), test_size=0.75, random_state=42)

In [20]:
train.shape

(58101, 55)

In [21]:
from random import randrange
from math import log
import random

def two_split(index, value, dataset):
    left = []
    right = []
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right


def entropy(dataset, classes):
    size = len(dataset)
    if size == 0:
        return 0
    ent = 0.0
    count = [0.0] * len(classes)
    for data in dataset:
        count[classes.index(data[-1])] += 1
    for c in count:
        proportion = c / size
        if proportion != 0:
            ent += -proportion * log(proportion, 2)
    return ent
        
    
    '''    
    for cls in classes:
        count = 0.0
        for data in dataset:
            if data[-1] == cls:
                count += 1;
        proportion = count / size
        if proportion != 0:
            ent += -proportion * log(proportion, 2)
    return ent'''


def entropy_compute(groups, classes):
    dataset = []
    for group in groups:
        dataset += group
    # information for this node
    ent_I = entropy(dataset, classes)
    # information required for subtrees / groups
    ent_remain = 0.0
    for group in groups:
        ent_temp = entropy(group, classes)
        ent_remain += (len(group)*1.0 / len(dataset)) * ent_temp
    return (ent_I - ent_remain)


def get_split(dataset, n_features):
    #X data, y label, n_feartures number of features you want
    features = [];
    classes = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 0, 0, None
    while b_groups is None:
        while len(features) < n_features:
            index = randrange(len(dataset[0]) - 1)
            if index not in features:
                features.append(index)
        for index in features:
            # get value range
            vals = set([row[index] for row in dataset])
            vals = list(vals)
            # if less than 10 classes, just use it
            # else compute 8 cuts
            if len(vals) > 10:
                low, high = min(vals), max(vals)
                diff = (high - low) / 21
                vals = [x * diff + low for x in range(0, 21)]
            for value in vals:
                groups = two_split(index, value, dataset)
                ent = entropy_compute(groups, classes)
                if ent > b_score:
                    b_index, b_value, b_score, b_groups = \
                        index, value, ent, groups
        '''        
        for row in dataset:
            # for quicker choosing
            if random.random() < 0.99:
                continue
            groups = two_split(index, row[index], dataset)
            ent = entropy_compute(groups, classes)
            if ent > b_score:
                b_index, b_value, b_score, b_groups = \
                    index, row[index], ent, groups
        '''
        if b_groups == None:
            for index in features:
                # get value range
                vals = set([row[index] for row in dataset])
                vals = list(vals)
                # if less than 10 classes, just use it
                # else compute 8 cuts
                if len(vals) > 10:
                    low, high = min(vals), max(vals)
                    diff = (high - low) / 11
                    vals = [(x * diff + low) for x in range(0, 11)]
                print "here the values", vals
                for value in vals:
                    groups = two_split(index, value, dataset)
                    ent = entropy_compute(groups, classes)
                    if ent > b_score:
                        b_index, b_value, b_score, b_groups = \
                            index, value, ent, groups
                print "the entropy is", ent
    return {'index':b_index, 'value':b_value, 'groups': b_groups}

def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key = outcomes.count)

# creat child split
def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    #print "the depth is", depth
    # check terminate
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left
    classes = list(set(row[-1] for row in (left + right)))
    if len(left) <= min_size or entropy(left, classes) < 0.01:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
    # process right,
    if len(right) <= min_size or entropy(right, classes) < 0.01:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

# build a decision tree
def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root

# prediction
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']
        
# random sample -- bootstrap
def resample(dataset, ratio):
    sample = list()
    num = round(len(dataset) * ratio)
    while len(sample) < num:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

# predict
def bagging_predict(trees, row):
    decisions = [predict(tree, row) for tree in trees]
    return max(set(decisions), key = decisions.count)

# Random Forest
def random_forest(train, test, max_depth, min_size\
                 , sample_ratio, n_trees, n_features):
    trees = []
    for i in range(n_trees):
        samples = resample(train, sample_ratio)
        tree = build_tree(samples, max_depth, min_size, n_features)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return predictions


In [ ]:
# build tree with max_depth 5
trees = []
n_trees = 2
sample_ratio = 0.2
max_depth = 5
min_size = 10
n_features = 0.4 * len(train[0])

for i in range(n_trees):
    print("tree number " + str(i + 1))
    samples = resample(train, sample_ratio)
    
    tree = build_tree(samples, max_depth, min_size, n_features)
    trees.append(tree)

tree number 1


In [123]:
trees_full = []
n_trees = 100
sample_ratio = 0.2
max_depth = 100
min_size = 25
n_features = 0.4 * len(train[0])

for i in range(n_trees):
    print("tree number " + str(i + 1))
    samples = resample(train, sample_ratio)
    tree = build_tree(samples, max_depth, min_size, n_features)
    trees_full.append(tree)

55

In [36]:
# test predictions for trees depth 5
predictions = [bagging_predict(trees, row) for row in X_test];

from sklearn.metrics import confusion_matrix
import pylab as pl
accuracy = confusion_matrix(y_test,predictions)
print(accuracy)

[[38037 14637    15     0     1     0   316]
 [16286 53455  1016     0    19     0     0]
 [    0  1263  7646     0     0     0     0]
 [    0     1   650     0     0     0     0]
 [    0  2209    70     0   194     0     0]
 [    0  1222  3151     0     0     0     0]
 [ 3713    24     0     0     0     0  1328]]


In [ ]:
# predictions for trees full
predictions2 = [bagging_predict(trees_full, row) for row in X_test];

accuracy = confusion_matrix(y_test,predictions2)
print(accuracy)

In [ ]:
# plot error rate
# error rate method
def get_error_rate(pred, Y):
    return sum(pred != Y) / float(len(Y))
# plot the error rate
def plot_error_rate2(er_train, er_test):
    df_error = pd.DataFrame([er_train, er_test]).T
    df_error.columns = ['Training', 'Test']
    plot1 = df_error.plot(linewidth = 3, figsize = (8,6),
            color = ['lightblue', 'darkblue'], grid = True)
    plot1.set_xlabel('Number of iterations', fontsize = 12)
    plot1.set_xticklabels(range(0,100,20))
    plot1.set_ylabel('Error rate', fontsize = 12)
    plot1.set_title('Error rate vs number of iterations', fontsize = 16)
    plt.axhline(y=er_test[0], linewidth=1, color = 'red', ls = 'dashed')

In [ ]:
# investigate the number of trees
nums = [1, 20, 40, 60, 80, 100]
err_train = []
err_test = []
for n in nums:
    # compute number of errors in each number of trees
    pred_train = [bagging_predict(trees[0:n], row) for row in train]
    pred_test = [bagging_predict(trees[0:n], row) for row in test]
    err_train.append(get_error_rate(pred_train, train[:, -1]))
    err_test.append(get_error_rate(pred_test, test[:, -1]))
plot_error_rate2(err_train, err_test)
plt.show()

In [ ]:
# investigate the number of trees
nums = [1, 20, 40, 60, 80, 100]
err_train = []
err_test = []
for n in nums:
    # compute number of errors in each number of trees
    pred_train = [bagging_predict(trees_full[0:n], row) for row in train]
    pred_test = [bagging_predict(trees_full[0:n], row) for row in test]
    err_train.append(get_error_rate(pred_train, train[:, -1]))
    err_test.append(get_error_rate(pred_test, test[:, -1]))
plot_error_rate2(err_train, err_test)
plt.show()

In [ ]:
def get_error_rate(pred, Y):
    return sum(pred != Y) / float(len(Y))

#print the error rate
def print_error_rate(err):
    print 'Error rate: Training: %.4f - Test: %.4f' % err

#get the error rate for train and test
def generic_clf(X_train, y_train, X_test, y_test, clf):
    clf=clf.fit(X_train,y_train.reshape(-1,1))
    pred_train = clf.predict(X_train)
    pred_test = clf.predict(X_test)
    return get_error_rate(pred_train, y_train), \
           get_error_rate(pred_test, y_test)
    

In [ ]:
def rank_features(dataset):
    info_gain = [];
    classes = list(set(row[-1] for row in dataset))
    features = range(len(dataset[0]))
    for index in features:            
        # get value range
        print "feature: ", index
        vals = set([row[index] for row in dataset])
        vals = list(vals)
        if len(vals) > 25:
            lo = min(vals)
            hi = max(vals)
            diff = (hi - lo) / 50.0
            vals = [x*diff + lo for x in range(1,50)]
        max_ent = 0;
        for row in dataset:
            for value in vals:
                groups = two_split(index, value, dataset)
                ent = entropy_compute(groups, classes)
                if ent > max_ent:
                    max_ent = ent
        info_gain.append(max_ent)
    feat_gain = [(i, info_gain[i]) for i in range(len(features))]
    feat_gain_sorted = sorted(feat_gain, key= lambda tup: tup[1])
    print "feature - information gain"
    for row in feat_gain_sorted:
        print row
    return feat_gain_sorted

In [ ]:
rank_features(test[0:100])

In [ ]:
feat_gain_sorted=rank_features1(train[0:1000])

In [ ]:
feat_gain_sorted=feat_gain_sorted[1:-1]

In [ ]:
bb=[]
cc=[]
for i in range(0,53):
    bb.append(temp[i][0])
    cc.append(temp[i][1])
print bb,cc

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
 

y_pos = np.arange(len(bb))    
plt.figure(figsize=(12,6))
plt.bar(y_pos, cc, align='center', alpha=0.5)
plt.xticks(y_pos, bb)
plt.ylabel('Feature importance')
plt.title('Feature importance')
 
plt.show()

In [ ]:
feat_gain_sorted=feat_gain_sorted[1:-1]
temp = sorted(feat_gain_sorted, key= lambda tup: tup[1],reverse=True)
bb=[]
cc=[]
for i in range(0,20):
    bb.append(temp[i][0])
    cc.append(temp[i][1])
print bb,cc
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
 

y_pos = np.arange(len(bb))    
plt.figure(figsize=(12,6))
plt.bar(y_pos, cc, align='center', alpha=0.5)
plt.xticks(y_pos, bb)
plt.ylabel('Feature importance')
plt.title('Feature importance')
 
plt.show()

In [ ]:
import math
def adaboost_clf2(y_train, X_train, y_test, X_test, M, Max_depth, Min_split):
    n_train, n_test = len(X_train), len(X_test)
    err_test, err_train = [], []
    classes = list(set(y_train))
    n_class = len(classes)
    # initialize weights
    w = np.ones(n_train) / n_train
    pred_train, pred_test = np.zeros((n_train, n_class)), np.zeros((n_test, n_class))
    clf = DecisionTreeClassifier(max_depth=Max_depth, min_samples_split=Min_split, random_state=42)
    for i in range(M):
        
        clf.fit(X_train, y_train, sample_weight = w)
        pred_train_i = clf.predict(X_train)
        pred_test_i = clf.predict(X_test)
        
        miss1 = [int(x) for x in (pred_train_i != y_train)]
        err_m = np.dot(w,miss1) / sum(w)
        #print err_m
        alpha_m = np.log((1-err_m) / float(err_m)) + math.log(n_class - 1)
        #print alpha_m
        
        miss2 = [x if x == 1 else -1 for x in miss1]
        w = np.multiply(w, np.exp([float(x) * alpha_m for x in miss2]))
        
        #print miss1
        #print miss2
        #print w
        # update
        for i in range(n_train):
            cls = pred_train_i[i]
            idx = classes.index(cls)
            pred_train[i, idx] += alpha_m
        for i in range(n_test):
            cls = pred_test_i[i]
            idx = classes.index(cls)
            pred_test[i, idx] += alpha_m
                                     
        prediction_train = [classes[np.argmax(row)] for row in pred_train]
        #print 'the prediction', prediction_train
        #print 'the y train', y_train
        prediction_test = [classes[np.argmax(row)] for row in pred_test]
        err_train.append(get_error_rate(prediction_train, y_train))
        err_test.append(get_error_rate(prediction_test, y_test))
    return err_train, err_test,prediction_test



In [ ]:
err_train, err_test,prediction_test=adaboost_clf2(y_train, X_train, y_test, X_test, 100, 1, 2)

In [ ]:
def plot_error_rate(er_train, er_test):
    df_error = pd.DataFrame([er_train, er_test]).T
    df_error.columns = ['Training', 'Test']
    plot1 = df_error.plot(linewidth = 3, figsize = (8,6),
            color = ['lightblue', 'darkblue'], grid = True)
    plot1.set_xlabel('Number of iterations', fontsize = 12)
    plot1.set_xticklabels(range(0,500,20))
    plot1.set_ylabel('Error rate', fontsize = 12)
    plot1.set_title('Error rate vs number of iterations', fontsize = 16)
    plt.axhline(y=er_test[0], linewidth=1, color = 'red', ls = 'dashed

In [ ]:
accuracy1 = confusion_matrix(y_test,prediction_test)

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
labels = ['1', '2','3','4','5','6','7']
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(accuracy1, classes=labels,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(accuracy1, classes=labels,normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
X_train=mnist_train_df.ix[1:785,1:].T.values
y_train=mnist_train_df.ix[0,1:].values
X_test=mnist_train_df.ix[1:785,1:].T.values
y_test=mnist_train_df.ix[0,1:].values

In [ ]:
err_train, err_test,prediction_test=adaboost_clf2(y_train, X_train, y_test, X_test, 100, 10, 2)

In [ ]:
plot_error_rate(err_train, err_test)